In [ ]:
import asyncio
import pandas as pd
from datetime import datetime, timedelta
from ib_insync import IB, util, Order, Contract

# --- 1. 规则引擎定义 (Strategy Pattern) ---

class RiskRule:
    def check(self, trade, fills_df):
        return {"is_allowed": True, "reason": ""}

class PositionLimitRule(RiskRule):
    
    def __init__(self, limits):
        self.limits = limits
        
    def check(self, trade, fills_df):
        symbol = trade.contract.symbol
        qty = trade.order.totalQuantity
        if symbol not in self.limits:
            return {"is_allowed": False, "reason": f"品种 {symbol} 不在白名单中"}
        if qty > self.limits[symbol]:
            return {"is_allowed": False, "reason": f"{symbol} 仓位限制 {self.limits[symbol]}，试图开 {qty}"}
        return {"is_allowed": True}

class ContinuousLossRule(RiskRule):
    
    def __init__(self, max_cnt=3, hours=12):
        self.max_cnt, self.hours = max_cnt, hours
    
    def check(self, trade, fills_df):
        symbol = trade.contract.symbol
        time_limit = datetime.now() - timedelta(hours=self.hours)
        symbol_fills = fills_df[(fills_df['symbol'] == symbol) & (fills_df['ny_time'] >= time_limit)]
        if len(symbol_fills) >= self.max_cnt:
            last_pnls = symbol_fills.head(self.max_cnt)['pnl'].tolist()
            if all(p < 0 for p in last_pnls):
                return {"is_allowed": False, "reason": f"{symbol} 在{self.hours}小时内连续止损{self.max_cnt}次"}
        return {"is_allowed": True}

class DailyLossLimitRule(RiskRule):
    
    def __init__(self, limit=-200, hours=24):
        self.limit, self.hours = limit, hours
    
    def check(self, trade, fills_df):
        time_limit = datetime.now() - timedelta(hours=self.hours)
        total_pnl = fills_df[fills_df['ny_time'] >= time_limit]['pnl'].sum()
        if total_pnl <= self.limit:
            return {"is_allowed": False, "reason": f"24h累计亏损 {total_pnl} 已超限额 {self.limit}"}
        return {"is_allowed": True}

# --- 2. 核心管理器 ---

class RiskControlManager:
    def __init__(self, ib):
        self.ib = ib
        self.processed_ids = set()
        # 注册规则
        self.rules = [
            PositionLimitRule({'MES': 2, 'NK225MC': 6}),
            ContinuousLossRule(max_cnt=3, hours=96),
            DailyLossLimitRule(limit=-200, hours=96)
        ]

    def get_recent_fills(self):
        fills = self.ib.fills()
        if not fills: return pd.DataFrame(columns=['symbol', 'ny_time', 'pnl'])
        df = util.df(fills)
        df['symbol'] = df['contract'].apply(lambda x: x.symbol)
        df['ny_time'] = pd.to_datetime(df['execution'].apply(lambda x: x.time)).dt.tz_convert('US/Eastern').dt.tz_localize(None)
        df['pnl'] = df['commissionReport'].apply(lambda x: x.realizedPNL)
        return df[df['pnl'] != 0].sort_values(by='ny_time', ascending=False)

    def onOpenOrder(self, trade):
        p_id = trade.order.permId
        if p_id in self.processed_ids: return
        
        # 只拦截待成交状态
        if trade.orderStatus.status not in ['PreSubmitted', 'Submitted']: return

        fills_df = self.get_recent_fills()
        for rule in self.rules:
            result = rule.check(trade, fills_df)
            if not result['is_allowed']:
                print(f"🚨 [拦截] {trade.contract.symbol} | 原因: {result['reason']}")
                self.processed_ids.add(p_id)
                asyncio.create_task(self.safe_cancel(p_id))
                return

        self.processed_ids.add(p_id)
        print(f"✅ [通过] {trade.contract.symbol} (PermID: {p_id})")

    async def safe_cancel(self, p_id):
        # 强制同步订单以获取有效的 OrderId
        self.ib.reqAutoOpenOrders(True)
        # 给 TWS 0.5秒同步时间
        await asyncio.sleep(0.5)
        trades = self.ib.reqOpenOrders()
        target = next((t for t in trades if t.order.permId == p_id and t.order.orderId != 0), None)
        if target:
            self.ib.cancelOrder(target.order)
            print(f"🗑️ 成功撤单: {p_id}")

# --- 3. 运行入口 (Main Loop) ---

async def main():
    ib = IB()
    manager = RiskControlManager(ib)
    
    # 绑定事件
    ib.openOrderEvent += manager.onOpenOrder

    while True:
        try:
            if not ib.isConnected():
                print("📡 正在连接 TWS (127.0.0.1:7497)...")
                # clientId=0 是认领外部订单的关键
                await ib.connectAsync('127.0.0.1', 7497, clientId=0)
                ib.reqAutoOpenOrders(True) # 开启自动接管
                print("🚀 风控系统已启动并连接成功")
            
            await asyncio.sleep(10) # 维持心跳

        except Exception as e:
            print(f"⚠️ 连接异常: {e}")
            await asyncio.sleep(5)

if __name__ == "__main__":
    # 启动异步循环
    util.patchAsyncio()
    try:
        asyncio.run(main())
    except KeyboardInterrupt:
        print("👋 系统关闭")

📡 正在连接 TWS (127.0.0.1:7497)...
🚀 风控系统已启动并连接成功
🚨 [拦截] MES | 原因: MES 在96小时内连续止损3次
🗑️ 成功撤单: 818802749
🚨 [拦截] MGC | 原因: 品种 MGC 不在白名单中
🗑️ 成功撤单: 818802751
🚨 [拦截] MES | 原因: MES 仓位限制 2，试图开 3.0
🗑️ 成功撤单: 818802752
🚨 [拦截] N225MC | 原因: 品种 N225MC 不在白名单中
🗑️ 成功撤单: 818802753
